In [77]:
import os  
import pickle
import pandas as pd
os.chdir(r'C:\Users\limso\Downloads')
df =pd.read_pickle('ModelOneHots')

In [78]:
lab_all = df['myerTypes']
df_all = df.drop(['I(1)/E','N(1)/S','P(1)/J','F(1)/T','myerTypes'], axis = 1)
lab_IE = df['I(1)/E']
df_IE = df.drop(['I(1)/E','N(1)/S','P(1)/J','F(1)/T','myerTypes'], axis = 1)
lab_NS = df['N(1)/S']
df_NS = df.drop(['I(1)/E','N(1)/S','P(1)/J','F(1)/T','myerTypes'], axis = 1)
lab_PJ = df['P(1)/J']
df_PJ = df.drop(['I(1)/E','N(1)/S','P(1)/J','F(1)/T','myerTypes'], axis = 1)
lab_FT = df['F(1)/T']
df_FT = df.drop(['I(1)/E','N(1)/S','P(1)/J','F(1)/T','myerTypes'], axis = 1)


In [70]:
#Model Building

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from sklearn import preprocessing
from keras import optimizers
from keras.layers import Dropout
from keras.layers import Conv1D

min_max_scaler = preprocessing.MinMaxScaler()
def scaler(dataframe):
    df_scaled = min_max_scaler.fit_transform(dataframe)
    df_normalized = pd.DataFrame(df_scaled)
    return df_normalized

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=314))
model.add(tf.keras.layers.Embedding(314,314))
model.add(tf.keras.layers.Conv1D((64),3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LeakyReLU())
model.add(tf.keras.layers.Dense(16, activation='softmax'))
model.add(tf.keras.layers.Dropout(0.02))
sgd=keras.optimizers.SGD(momentum=0.9,decay=1e-6,lr=0.001, nesterov=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)
print(model.summary())

#Model for single letter
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.Input(shape=314))
model2.add(tf.keras.layers.Embedding(314,314))
model2.add(tf.keras.layers.Conv1D((64),3))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(32))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.LeakyReLU())
model2.add(tf.keras.layers.Dense(2, activation='softmax')) #Only change this
model2.add(tf.keras.layers.Dropout(0.02))
sgd=keras.optimizers.SGD(momentum=0.9,decay=1e-6,lr=0.001, nesterov=True)
model2.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd)
print(model2.summary())


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 314, 314)          98596     
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 312, 64)           60352     
_________________________________________________________________
flatten_15 (Flatten)         (None, 19968)             0         
_________________________________________________________________
dense_30 (Dense)             (None, 32)                639008    
_________________________________________________________________
batch_normalization_15 (Batc (None, 32)                128       
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 32)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 16)              

In [74]:
#Run 4 letters model

types = ['ISTJ','ISTP','ISFJ','ISFP','INFJ','INFP','INTJ','INTP','ESTP','ESTJ','ESFP','ESFJ','ENFP','ENFJ','ENTP','ENTJ']
typedict = dict.fromkeys(types,0)
for i,k in enumerate(typedict):
    typedict[k] = i
lst = []
for i in lab_all:
    lst.append(typedict.get(i))
from keras.utils import to_categorical
lab_all = to_categorical(lst)

df_all = scaler(df_all)
history_all = model.fit(x=df_all,y=lab_all,batch_size=64,epochs=12,validation_split=0.2)


Train on 6940 samples, validate on 1735 samples
Epoch 1/12
6940/6940 [==============================] - 21s 3ms/sample - loss: 2.7339 - accuracy: 0.2659 - val_loss: 2.5849 - val_accuracy: 0.4179
Epoch 2/12
6940/6940 [==============================] - 19s 3ms/sample - loss: 2.1493 - accuracy: 0.4831 - val_loss: 2.2899 - val_accuracy: 0.4386
Epoch 3/12
6940/6940 [==============================] - 19s 3ms/sample - loss: 1.9362 - accuracy: 0.5081 - val_loss: 2.0038 - val_accuracy: 0.4484
Epoch 4/12
6940/6940 [==============================] - 19s 3ms/sample - loss: 1.8553 - accuracy: 0.5210 - val_loss: 1.8374 - val_accuracy: 0.4467
Epoch 5/12
6940/6940 [==============================] - 18s 3ms/sample - loss: 1.7875 - accuracy: 0.5246 - val_loss: 1.8047 - val_accuracy: 0.4524
Epoch 6/12
6940/6940 [==============================] - 19s 3ms/sample - loss: 1.7503 - accuracy: 0.5353 - val_loss: 1.8068 - val_accuracy: 0.4548
Epoch 7/12
6940/6940 [==============================] - 19s 3ms/sample

In [75]:
#Accuracy Metrics for 4 letter model
from plot_keras_history import plot_history
plot_history(history_all, path="interpolated_all.png", interpolate=True)
plt.close()

In [79]:
#I/E Model
lab_IE = to_categorical(lab_IE)
df_IE = scaler(df_IE)
history_IE = model2.fit(x=df_IE,y=lab_IE,batch_size=64,epochs=12,validation_split=0.2)


Train on 6940 samples, validate on 1735 samples
Epoch 1/12
6940/6940 [==============================] - 19s 3ms/sample - loss: 0.5003 - accuracy: 0.8388 - val_loss: 0.4642 - val_accuracy: 0.7908
Epoch 2/12
6940/6940 [==============================] - 18s 3ms/sample - loss: 0.4938 - accuracy: 0.8374 - val_loss: 0.4730 - val_accuracy: 0.7885
Epoch 3/12
6940/6940 [==============================] - 19s 3ms/sample - loss: 0.4812 - accuracy: 0.8442 - val_loss: 0.4688 - val_accuracy: 0.7896
Epoch 4/12
6940/6940 [==============================] - 17s 2ms/sample - loss: 0.4832 - accuracy: 0.8482 - val_loss: 0.4700 - val_accuracy: 0.7764
Epoch 5/12
6940/6940 [==============================] - 8s 1ms/sample - loss: 0.4826 - accuracy: 0.8519 - val_loss: 0.4756 - val_accuracy: 0.7902
Epoch 6/12
6940/6940 [==============================] - 9s 1ms/sample - loss: 0.4528 - accuracy: 0.8579 - val_loss: 0.4743 - val_accuracy: 0.7856
Epoch 7/12
6940/6940 [==============================] - 9s 1ms/sample - 

In [80]:
#Accuracy Metrics for I/E model
plot_history(history_IE, path="interpolated_IE.png", interpolate=True)
plt.close()

In [81]:
#N/S Model
lab_NS = to_categorical(lab_NS)
df_NS = scaler(df_NS)
history_NS = model2.fit(x=df_NS,y=lab_NS,batch_size=64,epochs=12,validation_split=0.2)
plot_history(history_NS, path="interpolated_NS.png", interpolate=True)
plt.close()


Train on 6940 samples, validate on 1735 samples
Epoch 1/12
6940/6940 [==============================] - 7s 1ms/sample - loss: 0.8184 - accuracy: 0.7659 - val_loss: 0.4707 - val_accuracy: 0.8144
Epoch 2/12
6940/6940 [==============================] - 7s 1ms/sample - loss: 0.6642 - accuracy: 0.7927 - val_loss: 0.5303 - val_accuracy: 0.7862
Epoch 3/12
6940/6940 [==============================] - 7s 1ms/sample - loss: 0.6273 - accuracy: 0.8182 - val_loss: 0.4406 - val_accuracy: 0.8173
Epoch 4/12
6940/6940 [==============================] - 8s 1ms/sample - loss: 0.5320 - accuracy: 0.8434 - val_loss: 0.4073 - val_accuracy: 0.8565
Epoch 5/12
6940/6940 [==============================] - 10s 1ms/sample - loss: 0.5237 - accuracy: 0.8550 - val_loss: 0.3936 - val_accuracy: 0.8565
Epoch 6/12
6940/6940 [==============================] - 12s 2ms/sample - loss: 0.5113 - accuracy: 0.8569 - val_loss: 0.3889 - val_accuracy: 0.8571
Epoch 7/12
6940/6940 [==============================] - 13s 2ms/sample - l

In [82]:
#P/J Model
lab_PJ = to_categorical(lab_PJ)
df_PJ = scaler(df_PJ)
history_PJ = model2.fit(x=df_PJ,y=lab_PJ,batch_size=64,epochs=12,validation_split=0.2)
plot_history(history_PJ, path="interpolated_PJ.png", interpolate=True)
plt.close()


Train on 6940 samples, validate on 1735 samples
Epoch 1/12
6940/6940 [==============================] - 7s 1ms/sample - loss: 0.9160 - accuracy: 0.6082 - val_loss: 0.6409 - val_accuracy: 0.6300
Epoch 2/12
6940/6940 [==============================] - 8s 1ms/sample - loss: 0.7578 - accuracy: 0.6608 - val_loss: 0.6066 - val_accuracy: 0.6646
Epoch 3/12
6940/6940 [==============================] - 10s 1ms/sample - loss: 0.7116 - accuracy: 0.6975 - val_loss: 0.5953 - val_accuracy: 0.6755
Epoch 4/12
6940/6940 [==============================] - 12s 2ms/sample - loss: 0.6982 - accuracy: 0.7100 - val_loss: 0.5929 - val_accuracy: 0.6784
Epoch 5/12
6940/6940 [==============================] - 14s 2ms/sample - loss: 0.7110 - accuracy: 0.7169 - val_loss: 0.5890 - val_accuracy: 0.6749
Epoch 6/12
6940/6940 [==============================] - 15s 2ms/sample - loss: 0.6832 - accuracy: 0.7256 - val_loss: 0.5893 - val_accuracy: 0.6813
Epoch 7/12
6940/6940 [==============================] - 16s 2ms/sample -

In [83]:
#F/T Model
lab_FT = to_categorical(lab_FT)
df_FT = scaler(df_FT)
history_FT = model2.fit(x=df_FT,y=lab_FT,batch_size=64,epochs=12,validation_split=0.2)
plot_history(history_FT, path="interpolated_FT.png", interpolate=True)
plt.close()


Train on 6940 samples, validate on 1735 samples
Epoch 1/12
6940/6940 [==============================] - 8s 1ms/sample - loss: 0.7908 - accuracy: 0.6325 - val_loss: 0.5610 - val_accuracy: 0.7153
Epoch 2/12
6940/6940 [==============================] - 13s 2ms/sample - loss: 0.6663 - accuracy: 0.7517 - val_loss: 0.5100 - val_accuracy: 0.7533
Epoch 3/12
6940/6940 [==============================] - 13s 2ms/sample - loss: 0.6057 - accuracy: 0.7745 - val_loss: 0.5030 - val_accuracy: 0.7539
Epoch 4/12
6940/6940 [==============================] - 13s 2ms/sample - loss: 0.5987 - accuracy: 0.7823 - val_loss: 0.5053 - val_accuracy: 0.7539
Epoch 5/12
6940/6940 [==============================] - 13s 2ms/sample - loss: 0.6041 - accuracy: 0.7818 - val_loss: 0.5092 - val_accuracy: 0.7522
Epoch 6/12
6940/6940 [==============================] - 14s 2ms/sample - loss: 0.5812 - accuracy: 0.7919 - val_loss: 0.5064 - val_accuracy: 0.7550
Epoch 7/12
6940/6940 [==============================] - 14s 2ms/sample 